# Notes, 1/3/18
* ODB website: https://software.ecmwf.int/wiki/display/ODBAPI/ODB+API+Home
* ODB API is a software developed at ECMWF for encoding and processing of observational data. It includes a SQL filtering and statistics engine, command line tools and APIs for C/C++, Fortran and Python. ODB API works with data format used in ECMWF observational feedback archive. Development of ODB API has been partially funded by the Met Office.
    * C/C++ base with Fortran and Python wrappers
* ODB examples seem to indicate that the API
    * Is for general SQL table creation and access
    * Tables have heterogeneous data in columns going across a row
        * Cumbersome to accees in C/C++/Fortran
        * Not so bad in Python (example uses pandas which makes this easy)
* ODB provides a command line tool called "odb"
    * SQL access, plotting tools, inspect/compare odb files
    * This seems more oriented toward observation data
* ODB files follow a convention for metadata
    * ODB Governance (?)

## Download and Install
* Download: https://software.ecmwf.int/wiki/display/ODBAPI/Releases
* Install instructions: https://software.ecmwf.int/wiki/display/ODBAPI/ODB+API+Installation
    * Build and install appear to work
        * Sample program in install/bin directory: odbql_c_example 
            * creates an odb file without error
            * "odb ls odb_file" shows its contents
    * Test works, but 4 tests fail
        * Output from ctest:
        
            > 99% tests passed, 4 tests failed out of 333  
            >
            > Label Time Summary:  
            > download_data     =   1.43 sec (6 tests)  
            > eckit             =  80.20 sec (96 tests)  
            > executable        =  82.10 sec (103 tests)  
            > metkit            =   0.46 sec (3 tests)  
            > mpi               =   6.66 sec (2 tests)  
            > odb_api           = 144.56 sec (234 tests)  
            > odb_api_ecml      =  56.02 sec (27 tests)  
            > odb_api_netcdf    =   0.56 sec (2 tests)  
            > script            = 141.70 sec (224 tests)  
            >
            > Total Test time (real) = 226.30 sec 
            >
            > The following tests FAILED:  
            >     41 - eckit_test_mpi (Failed)  
            >     42 - eckit_test_mpi_addcomm (Failed)  
            >     95 - ECKIT-221.sh (Failed)  
            >     96 - ECKIT-166.sh (Failed)  
            > Errors while running CTest  

    * The example program worked, so not sure what is broken.

        

# Meeting with Yannick, 1/5/18
* The ODB file is probably an SQL file
    * Developers likely took an open source API for SQL and wired it into ODB
* Are there templates for obs type tables in SQL?
    * I was looking for the C/C++/Fortran APIs to have something like pandas in Python
    * Yannick said that the tendancy is to design a table once, when a new instrument appears, then use that code over and over
        * The creation of a new table occurs very infrequently
        * Proablay not worth the effort to provide a generic table capability like pandas
* The 4 test failures are probably okay for now
    * MPI test failures may be indicating that I don't have MPI installed/enabled on my iMac
* We will potentially collaborate with ECMWF on development
    * Yannick will talk with people at AMS next week
    * Depending on how these talks go
        * Might get someone from ECMWF to come out here for a week or so to train us
        * Or, might go to ECMWF for their annual training session
* Use the website and code repository for help with API, examples, etc.
    * If this doesn't work well, perhaps the training will happen